In [ ]:
from polartoolkit import fetch, maps, profiles, regions, utils
#import polartoolkit as ptk
import verde as vd
import invert4geom
import numpy as np
import pandas as pd
import harmonica
import matplotlib.pyplot as plt

import xarray as xr

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

os.environ["POLARTOOLKIT_HEMISPHERE"] = "south"

region = regions.getz_ice_shelf

In [ ]:
Data = pd.read_csv('constraint_points.csv')
#Data = Data[:10000]
constraint_points = Data[0::1] #uses every 10th datapoint
constraint_points

In [ ]:

# block median with 1km blocks
reducer = vd.BlockReduce(reduction=np.median, spacing=2e3)
coordinates, data = reducer.filter(
    (constraint_points.easting, constraint_points.northing),
    constraint_points.bed_elevation
)

constraint_points = pd.DataFrame({"easting": coordinates[0], "northing": coordinates[1],"bed_elevation":data})
constraint_points

In [ ]:
spline = vd.Spline(damping=None)#0.001)



spline.fit(coordinates=(constraint_points.easting, constraint_points.northing), data=constraint_points['bed_elevation'])

In [ ]:
grid = spline.grid(region= region, spacing = 2000, data_names = 'upward')

In [ ]:
 grid.upward.plot()# get bedmap3 bed topography grid


In [ ]:
# get bedmap3 bed topography grid
bed_topography = fetch.bedmap3(layer="bed", region = region, reference="ellipsoid")


In [ ]:
_=utils.grd_compare(grid.upward, bed_topography, coast = True, 
                   grid1_name = 'Interpolated',
                    grid2_name = 'Bedmap3'
                   )

# Mask with xarray

In [ ]:
Ice_base = fetch.bedmap3(layer="icebase", region = region, spacing=2000, reference="ellipsoid",)

Ice_base = Ice_base.rename({"x":"easting", "y":"northing"})

In [ ]:
grid.upward

In [ ]:
Ice_base


In [ ]:
#interpolated_bed = grid.upward.where(grid.upward <= Ice_base, Ice_base)
interpolated_bed = xr.where(grid.upward > Ice_base, Ice_base, grid.upward)


In [ ]:
interpolated_bed.plot()

In [ ]:
(grid.upward - interpolated_bed).plot()

In [ ]:
interpolated_bed.to_netcdf('interpolated_bed.nc')